In [36]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import keras as kr
import matplotlib.pyplot as plt


In [24]:
biden_v_trump24 = pd.read_csv('../election_resources/biden_v_trump24.csv')
biden_v_trump24.head()

,samplesize,party,results,favorability
0,1500.0,d,0,-22.33400
1,1178.0,d,0,-20.66730
2,1076.0,d,0,-12.66730
3,2000.0,d,0,-14.43379
4,1500.0,d,0,-29.66750


In [25]:
y = biden_v_trump24['results']

In [26]:
biden_v_trump24 = biden_v_trump24.drop(columns=['results'])

In [27]:
biden_v_trump24.columns

Index(['samplesize', 'party', 'favorability'], dtype='object')

In [28]:
biden_v_trump24 = pd.get_dummies(biden_v_trump24, dtype=float)
biden_v_trump24.head()

,samplesize,favorability,party_d,party_r
0,1500.0,-22.33400,1.0,0.0
1,1178.0,-20.66730,1.0,0.0
2,1076.0,-12.66730,1.0,0.0
3,2000.0,-14.43379,1.0,0.0
4,1500.0,-29.66750,1.0,0.0


In [29]:
biden_v_trump24.shape

(1662, 4)

In [30]:
nn_model = tf.keras.models.load_model('trained_model_president.h5')
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 42        
                                                                 
 dense_1 (Dense)             (None, 9)                 81        
                                                                 
 dense_2 (Dense)             (None, 17)                187       
                                                                 
 dense_3 (Dense)             (None, 3)                 57        
                                                                 
 dense_4 (Dense)             (None, 9)                 45        
                                                                 
 dense_5 (Dense)             (None, 1)                 10        
                                                                 
Total params: 422
Trainable params: 422
Non-trainable pa

In [31]:
scaler = StandardScaler()
X_scaler = scaler.fit(biden_v_trump24)
X_scaled = X_scaler.transform(biden_v_trump24)


In [32]:
X_scaled

array([[-0.12783023, -0.33801254,  1.00240964, -1.00240964],
       [-0.33224768, -0.22363611,  1.00240964, -1.00240964],
       [-0.39700104,  0.32535979,  1.00240964, -1.00240964],
       ...,
       [ 0.31655553, -0.49813406, -0.99759615,  0.99759615],
       [-0.12783023, -0.06350087, -0.99759615,  0.99759615],
       [ 0.31655553, -0.65826244, -0.99759615,  0.99759615]])

In [33]:
predictions = nn_model.predict(X_scaled)

52/52 [==============================] - 0s 3ms/step


In [34]:
predictions

array([[0.00549383],
       [0.00164679],
       [0.00220622],
       ...,
       [0.99316037],
       [0.9379417 ],
       [0.9863794 ]], dtype=float32)

In [40]:
predictions_df = pd.DataFrame(predictions, columns=['predictions'])

In [39]:
#auc_roc = roc_auc_score(y, predictions)
mse = mean_squared_error(y, predictions)
mae = mean_absolute_error(y, predictions)
#print(f'AUC-ROC score: {auc_roc}')
print(f'MSE score: {mse}')
print(f'MAE score: {mae}')

MSE score: 0.4858347182339896
MAE score: 0.5127093844075438


In [41]:
predictions_df['results']=results
predictions_df.head()

,predictions,results
0,0.005494,0
1,0.001647,0
2,0.002206,0
3,0.000069,0
4,0.039327,0


In [42]:
predictions_df.to_csv('predictions.csv', index=False)

In [43]:
predictions_df.head()

,predictions,results
0,0.005494,0
1,0.001647,0
2,0.002206,0
3,0.000069,0
4,0.039327,0
